In [1]:
import pandas as pd
from statsmodels.formula.api import ols
import statsmodels.api as sm

In [2]:
data = pd.read_csv('forms.csv').loc[:, 'perguntar para o entrevistador:':]

In [3]:
data.columns = ['tipo', 'animais', 'n_cervos', 'macaco', 'urso', 'cobra', 'dia_noite', 'sol', 'trilha', 'arvore', 'casa']

In [4]:
parsed = data.copy()

In [5]:
respostas_certas_demo = {
    'n_cervos': '3',
    'macaco': 'Não havia macaco no trajeto',
    'urso': 'Dormindo',
    'cobra': 'Não havia cobra no trajeto',
    'dia_noite': 'Dia',
    'sol': 'Sim',
    'trilha': 'Terra',
    'arvore': 'Pinheiro',
    'casa': 'Vermelha'
}

In [6]:
respostas_certas_video = {
    'n_cervos': '3',
    'macaco': 'Não havia macaco no trajeto',
    'urso': 'Dormindo',
    'cobra': 'Não havia cobra no trajeto',
    'dia_noite': 'Dia',
    'sol': 'Não',
    'trilha': 'Terra',
    'arvore': 'Pinheiro',
    'casa': 'Vermelha'
}

In [7]:
# change to 1 when the answer is correct and 0 when it is wrong
# based on tipo column to check the right answer
for i in range(len(parsed)):
    if parsed.loc[i, 'tipo'] == 'demo':
        respostas = respostas_certas_demo
    else:
        respostas = respostas_certas_video  

    for key, value in respostas.items():
        if parsed.loc[i, key] == value:
            parsed.loc[i, key] = 1
        elif parsed.loc[i, key] == 'Não sei':
            parsed.loc[i, key] = 0
        else:
            parsed.loc[i, key] = 0

    if all(animal in parsed.loc[i, 'animais'] for animal in ['Cervo', 'Urso', 'Passarinho']):
        if len(parsed.loc[i, 'animais'].split(',')) == 3:
            parsed.loc[i, 'animais'] = 1
        else:
            parsed.loc[i, 'animais'] = 0
    elif any(animal in parsed.loc[i, 'animais'] for animal in ['Cervo', 'Urso', 'Passarinho']):
        parsed.loc[i, 'animais'] = 0
    else:
        parsed.loc[i, 'animais'] = 0

parsed.head()

,tipo,animais,n_cervos,macaco,urso,cobra,dia_noite,sol,trilha,arvore,casa
0,Demo,1,1,1,1,1,1,0,1,1,1
1,Vídeo,0,0,1,1,1,1,1,1,0,1
2,Demo,0,0,1,0,1,1,0,0,0,0
3,Vídeo,0,1,1,1,1,1,1,1,0,1
4,Demo,0,0,0,0,0,1,0,1,0,1


In [8]:
# convert all but tipo column to numeric
for col in parsed.columns[1:]:
    parsed[col] = pd.to_numeric(parsed[col])


In [9]:
respostas = [*parsed.animais, 
             *parsed.n_cervos,
             *parsed.macaco,
             *parsed.urso,
             *parsed.cobra,
             *parsed.dia_noite,
             *parsed.sol,
             *parsed.trilha,
             *parsed.arvore,
             *parsed.casa]

tipos = [*parsed.tipo] * 10

In [10]:
data = pd.DataFrame({'respostas': respostas, 'tipo': tipos})
data.head()

,respostas,tipo
0,1,Demo
1,0,Vídeo
2,0,Demo
3,0,Vídeo
4,0,Demo


In [11]:
model = ols('''respostas~ C(tipo, Sum)''', data=data).fit()

In [12]:
aov_table = sm.stats.anova_lm(model, typ=3)
print(aov_table)

                  sum_sq     df           F         PR(>F)
Intercept     218.565681    1.0  977.401605  1.561486e-119
C(tipo, Sum)    0.837681    1.0    3.746018   5.349922e-02
Residual      111.362319  498.0         NaN            NaN


In [471]:
# filter dataframe based on perguntar para o entrevistador: value

video = data[data.tipo == 'Vídeo']
demo = data[data.tipo == 'Demo']

In [472]:
total_demo = 0
total_video = 0

abstinencias_demo = 0
abstinencias_video = 0

In [473]:
def count_wrong_animals(df):
    animais = df.animais.str.split(', ')
    count = 0
    for resp in animais:
        #if all animal in resp
        if any(animal in resp for animal in ['Macaco', 'Cobra', 'Aranha']):
            count += 1
    return count

In [474]:
def extrict_animals(df, animals):
    animais = df.animais.str.split(', ')
    count = 0
    for resp in animais:
        #if all animal in resp
        if all(animal in resp for animal in animals) and len(animals) == len(resp):
            count += 1
    return count

# Pergunta 1:
## Quais animais apareceram no trajeto?

Resposta correta:
- Cervo, Passarinho, Urso

Resposta parcial:
- Contem algum dos animais da respsota correta, mas nao todos

In [475]:
# animais demo

total = len(demo)
respostas_certas = extrict_animals(demo, ["Cervo", "Passarinho", "Urso"])
respostas_erradas = count_wrong_animals(demo)
respostas_parciais = total - respostas_certas - respostas_erradas


total_demo += respostas_certas
print(f'Respostas totalmente corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Respostas parcialmente corretas: {respostas_parciais}')
print(f'frequencia de respostas parcialmente corretas: {respostas_parciais/total}')
print()
print(f'Respostas totalmente erradas: {respostas_erradas}')
print(f'frequencia de respostas erradas: {respostas_erradas/total}')


AttributeError: 'DataFrame' object has no attribute 'animais'

In [ ]:
# animais video

total = len(video)
respostas_certas = extrict_animals(video, ["Cervo", "Passarinho", "Urso"])
respostas_erradas = count_wrong_animals(video)
respostas_parciais = total - respostas_certas - respostas_erradas


total_video += respostas_certas
print(f'Respostas totalmente corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Respostas parcialmente corretas: {respostas_parciais}')
print(f'frequencia de respostas parcialmente corretas: {respostas_parciais/total}')
print()
print(f'Respostas totalmente erradas: {respostas_erradas}')
print(f'frequencia de respostas erradas: {respostas_erradas/total}')

Respostas totalmente corretas: 6
frequencia de respostas corretas: 0.2608695652173913

Respostas parcialmente corretas: 12
frequencia de respostas parcialmente corretas: 0.5217391304347826

Respostas totalmente erradas: 5
frequencia de respostas erradas: 0.21739130434782608


# Pergunta 2
## Quanto Cervos tinha ao longo do trajeto?

Resposta correta:
- 3

Abstinencia:
- Não sei

In [ ]:
def count_right(df, column, value):
    return len(df[df[column] == value])

In [ ]:
# demo n_cervos

total = len(demo)
respostas_certas = count_right(demo, 'n_cervos', '3')
abstinencias = count_right(demo, 'n_cervos', 'Não sei')

total_demo += respostas_certas
abstinencias_demo += abstinencias
print(f'Respostas corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Abstinencias: {abstinencias}')
print(f'frequencia de abstinencias: {abstinencias/total}')
print()
print(f'Respostas erradas: {total - respostas_certas - abstinencias}')
print(f'frequencia de respostas erradas: {(total - respostas_certas - abstinencias)/total}')


Respostas corretas: 14
frequencia de respostas corretas: 0.5185185185185185

Abstinencias: 2
frequencia de abstinencias: 0.07407407407407407

Respostas erradas: 11
frequencia de respostas erradas: 0.4074074074074074


In [ ]:
# video n_cervos

total = len(video)
respostas_certas = count_right(video, 'n_cervos', '3')
abstinencias = count_right(video, 'n_cervos', 'Não sei')

total_video += respostas_certas
abstinencias_video += abstinencias
print(f'Respostas corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Abstinencias: {abstinencias}')
print(f'frequencia de abstinencias: {abstinencias/total}')
print()
print(f'Respostas erradas: {total - respostas_certas - abstinencias}')
print(f'frequencia de respostas erradas: {(total - respostas_certas - abstinencias)/total}')

Respostas corretas: 16
frequencia de respostas corretas: 0.6956521739130435

Abstinencias: 0
frequencia de abstinencias: 0.0

Respostas erradas: 7
frequencia de respostas erradas: 0.30434782608695654


# Pergunta 3
## Aonde estava o Macaco?

Resposta Certa:
- Não havia macaco no trajeto

Abstinencia:
- Não sei

In [ ]:
# Demo macaco

total = len(demo)
respostas_certas = count_right(demo, 'macaco', 'Não havia macaco no trajeto')
abstinencias = count_right(demo, 'macaco', 'Não sei')

total_demo += respostas_certas
abstinencias_demo += abstinencias
print(f'Respostas corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Abstinencias: {abstinencias}')
print(f'frequencia de abstinencias: {abstinencias/total}')
print()
print(f'Respostas erradas: {total - respostas_certas - abstinencias}')
print(f'frequencia de respostas erradas: {(total - respostas_certas - abstinencias)/total}')

Respostas corretas: 19
frequencia de respostas corretas: 0.7037037037037037

Abstinencias: 8
frequencia de abstinencias: 0.2962962962962963

Respostas erradas: 0
frequencia de respostas erradas: 0.0


In [ ]:
# Video macaco

total = len(video)
respostas_certas = count_right(video, 'macaco', 'Não havia macaco no trajeto')
abstinencias = count_right(video, 'macaco', 'Não sei')

total_video += respostas_certas
abstinencias_video += abstinencias
print(f'Respostas corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Abstinencias: {abstinencias}')
print(f'frequencia de abstinencias: {abstinencias/total}')
print()
print(f'Respostas erradas: {total - respostas_certas - abstinencias}')
print(f'frequencia de respostas erradas: {(total - respostas_certas - abstinencias)/total}')

Respostas corretas: 17
frequencia de respostas corretas: 0.7391304347826086

Abstinencias: 3
frequencia de abstinencias: 0.13043478260869565

Respostas erradas: 3
frequencia de respostas erradas: 0.13043478260869565


# Perugunta 4
## O que o urso estava fazendo?

Resposta:
- Dormindo

Abistinencia:
- Não sei

In [ ]:
# demo urso

total = len(demo)
respostas_certas = count_right(demo, 'urso', 'Dormindo')
abstinencias = count_right(demo, 'urso', 'Não sei')

total_demo += respostas_certas
abstinencias_demo += abstinencias
print(f'Respostas corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Abstinencias: {abstinencias}')
print(f'frequencia de abstinencias: {abstinencias/total}')
print()
print(f'Respostas erradas: {total - respostas_certas - abstinencias}')
print(f'frequencia de respostas erradas: {(total - respostas_certas - abstinencias)/total}')

Respostas corretas: 20
frequencia de respostas corretas: 0.7407407407407407

Abstinencias: 4
frequencia de abstinencias: 0.14814814814814814

Respostas erradas: 3
frequencia de respostas erradas: 0.1111111111111111


In [ ]:
# Video urso

total = len(video)
respostas_certas = count_right(video, 'urso', 'Dormindo')
abstinencias = count_right(video, 'urso', 'Não sei')

total_video += respostas_certas
abstinencias_video += abstinencias
print(f'Respostas corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Abstinencias: {abstinencias}')
print(f'frequencia de abstinencias: {abstinencias/total}')
print()
print(f'Respostas erradas: {total - respostas_certas - abstinencias}')
print(f'frequencia de respostas erradas: {(total - respostas_certas - abstinencias)/total}')

Respostas corretas: 21
frequencia de respostas corretas: 0.9130434782608695

Abstinencias: 0
frequencia de abstinencias: 0.0

Respostas erradas: 2
frequencia de respostas erradas: 0.08695652173913043


# Perugunta 5
## Qual a cor da cobra?

Resposta:
- Não havia cobra no trajeto

Abistinencia:
- Não sei

In [ ]:
# demo cobra

total = len(demo)
respostas_certas = count_right(demo, 'cobra', 'Não havia cobra no trajeto')
abstinencias = count_right(demo, 'cobra', 'Não sei')

total_demo += respostas_certas
abstinencias_demo += abstinencias
print(f'Respostas corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Abstinencias: {abstinencias}')
print(f'frequencia de abstinencias: {abstinencias/total}')
print()
print(f'Respostas erradas: {total - respostas_certas - abstinencias}')
print(f'frequencia de respostas erradas: {(total - respostas_certas - abstinencias)/total}')

Respostas corretas: 19
frequencia de respostas corretas: 0.7037037037037037

Abstinencias: 7
frequencia de abstinencias: 0.25925925925925924

Respostas erradas: 1
frequencia de respostas erradas: 0.037037037037037035


In [ ]:
# Video cobra

total = len(video)
respostas_certas = count_right(video, 'cobra', 'Não havia cobra no trajeto')
abstinencias = count_right(video, 'cobra', 'Não sei')

total_video += respostas_certas
abstinencias_video += abstinencias
print(f'Respostas corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Abstinencias: {abstinencias}')
print(f'frequencia de abstinencias: {abstinencias/total}')
print()
print(f'Respostas erradas: {total - respostas_certas - abstinencias}')
print(f'frequencia de respostas erradas: {(total - respostas_certas - abstinencias)/total}')

Respostas corretas: 19
frequencia de respostas corretas: 0.8260869565217391

Abstinencias: 4
frequencia de abstinencias: 0.17391304347826086

Respostas erradas: 0
frequencia de respostas erradas: 0.0


# Perugunta 6
## O percurso aconteceu durante o dia ou a noite?

Resposta:
- Dia

Abistinencia:
- Não sei

In [ ]:
# Demo dia_noite

total = len(demo)
respostas_certas = count_right(demo, 'dia_noite', 'Dia')
abstinencias = count_right(demo, 'dia_noite', 'Não sei')

total_demo += respostas_certas
abstinencias_demo += abstinencias
print(f'Respostas corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Abstinencias: {abstinencias}')
print(f'frequencia de abstinencias: {abstinencias/total}')
print()
print(f'Respostas erradas: {total - respostas_certas - abstinencias}')
print(f'frequencia de respostas erradas: {(total - respostas_certas - abstinencias)/total}')

Respostas corretas: 27
frequencia de respostas corretas: 1.0

Abstinencias: 0
frequencia de abstinencias: 0.0

Respostas erradas: 0
frequencia de respostas erradas: 0.0


In [ ]:
# video dia_noite

total = len(video)
respostas_certas = count_right(video, 'dia_noite', 'Dia')
abstinencias = count_right(video, 'dia_noite', 'Não sei')

total_video += respostas_certas
abstinencias_video += abstinencias
print(f'Respostas corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Abstinencias: {abstinencias}')
print(f'frequencia de abstinencias: {abstinencias/total}')
print()
print(f'Respostas erradas: {total - respostas_certas - abstinencias}')
print(f'frequencia de respostas erradas: {(total - respostas_certas - abstinencias)/total}')

Respostas corretas: 23
frequencia de respostas corretas: 1.0

Abstinencias: 0
frequencia de abstinencias: 0.0

Respostas erradas: 0
frequencia de respostas erradas: 0.0


# Perugunta 7
## Havia sol no céu?

Resposta:
- Sim -> demo
- Nao -> Video

Abistinencia:
- Não sei

In [ ]:
# demo sol

total = len(demo)
respostas_certas = count_right(demo, 'sol', 'Sim')
abstinencias = count_right(demo, 'sol', 'Não sei')

total_demo += respostas_certas
abstinencias_demo += abstinencias
print(f'Respostas corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Abstinencias: {abstinencias}')
print(f'frequencia de abstinencias: {abstinencias/total}')
print()
print(f'Respostas erradas: {total - respostas_certas - abstinencias}')
print(f'frequencia de respostas erradas: {(total - respostas_certas - abstinencias)/total}')

Respostas corretas: 8
frequencia de respostas corretas: 0.2962962962962963

Abstinencias: 15
frequencia de abstinencias: 0.5555555555555556

Respostas erradas: 4
frequencia de respostas erradas: 0.14814814814814814


In [ ]:
# video sol

total = len(video)
respostas_certas = count_right(video, 'sol', 'Não')
abstinencias = count_right(video, 'sol', 'Não sei')

total_video += respostas_certas
abstinencias_video += abstinencias
print(f'Respostas corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Abstinencias: {abstinencias}')
print(f'frequencia de abstinencias: {abstinencias/total}')
print()
print(f'Respostas erradas: {total - respostas_certas - abstinencias}')
print(f'frequencia de respostas erradas: {(total - respostas_certas - abstinencias)/total}')

Respostas corretas: 13
frequencia de respostas corretas: 0.5652173913043478

Abstinencias: 7
frequencia de abstinencias: 0.30434782608695654

Respostas erradas: 3
frequencia de respostas erradas: 0.13043478260869565


# Perugunta 8
## Do que era feita a trilha?

Resposta:
- Terra

Abistinencia:
- Não sei

In [ ]:
# demo trilha

total = len(demo)
respostas_certas = count_right(demo, 'trilha', 'Terra')
abstinencias = count_right(demo, 'trilha', 'Não sei')

total_demo += respostas_certas
abstinencias_demo += abstinencias
print(f'Respostas corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Abstinencias: {abstinencias}')
print(f'frequencia de abstinencias: {abstinencias/total}')
print()
print(f'Respostas erradas: {total - respostas_certas - abstinencias}')
print(f'frequencia de respostas erradas: {(total - respostas_certas - abstinencias)/total}')

Respostas corretas: 20
frequencia de respostas corretas: 0.7407407407407407

Abstinencias: 0
frequencia de abstinencias: 0.0

Respostas erradas: 7
frequencia de respostas erradas: 0.25925925925925924


In [ ]:
# video trilha

total = len(video)
respostas_certas = count_right(video, 'trilha', 'Terra')
abstinencias = count_right(video, 'trilha', 'Não sei')

total_video += respostas_certas
abstinencias_video += abstinencias
print(f'Respostas corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Abstinencias: {abstinencias}')
print(f'frequencia de abstinencias: {abstinencias/total}')
print()
print(f'Respostas erradas: {total - respostas_certas - abstinencias}')
print(f'frequencia de respostas erradas: {(total - respostas_certas - abstinencias)/total}')

Respostas corretas: 19
frequencia de respostas corretas: 0.8260869565217391

Abstinencias: 0
frequencia de abstinencias: 0.0

Respostas erradas: 4
frequencia de respostas erradas: 0.17391304347826086


# Perugunta 9
## Que tipo de árvore você mais viu?

Resposta:
- Pinheiro

Abistinencia:
- Não sei

In [ ]:
# demo arvore

total = len(demo)
respostas_certas = count_right(demo, 'arvore', 'Pinheiro')
abstinencias = count_right(demo, 'arvore', 'Não sei')

total_demo += respostas_certas
abstinencias_demo += abstinencias
print(f'Respostas corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Abstinencias: {abstinencias}')
print(f'frequencia de abstinencias: {abstinencias/total}')
print()
print(f'Respostas erradas: {total - respostas_certas - abstinencias}')
print(f'frequencia de respostas erradas: {(total - respostas_certas - abstinencias)/total}')

Respostas corretas: 9
frequencia de respostas corretas: 0.3333333333333333

Abstinencias: 10
frequencia de abstinencias: 0.37037037037037035

Respostas erradas: 8
frequencia de respostas erradas: 0.2962962962962963


In [ ]:
# video arvore

total = len(video)
respostas_certas = count_right(video, 'arvore', 'Pinheiro')
abstinencias = count_right(video, 'arvore', 'Não sei')

total_video += respostas_certas
abstinencias_video += abstinencias
print(f'Respostas corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Abstinencias: {abstinencias}')
print(f'frequencia de abstinencias: {abstinencias/total}')
print()
print(f'Respostas erradas: {total - respostas_certas - abstinencias}')
print(f'frequencia de respostas erradas: {(total - respostas_certas - abstinencias)/total}')

Respostas corretas: 6
frequencia de respostas corretas: 0.2608695652173913

Abstinencias: 5
frequencia de abstinencias: 0.21739130434782608

Respostas erradas: 12
frequencia de respostas erradas: 0.5217391304347826


# Perugunta 10
## Qual a cor da casa?

Resposta:
- Vermelha

Abistinencia:
- Não sei

In [ ]:
# demo casa

total = len(demo)
respostas_certas = count_right(demo, 'casa', 'Vermelha')
abstinencias = count_right(demo, 'casa', 'Não sei')

total_demo += respostas_certas
abstinencias_demo += abstinencias
print(f'Respostas corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Abstinencias: {abstinencias}')
print(f'frequencia de abstinencias: {abstinencias/total}')
print()
print(f'Respostas erradas: {total - respostas_certas - abstinencias}')
print(f'frequencia de respostas erradas: {(total - respostas_certas - abstinencias)/total}')

Respostas corretas: 23
frequencia de respostas corretas: 0.8518518518518519

Abstinencias: 0
frequencia de abstinencias: 0.0

Respostas erradas: 4
frequencia de respostas erradas: 0.14814814814814814


In [ ]:
# video casa

total = len(video)
respostas_certas = count_right(video, 'casa', 'Vermelha')
abstinencias = count_right(video, 'casa', 'Não sei')

total_video += respostas_certas
abstinencias_video += abstinencias
print(f'Respostas corretas: {respostas_certas}')
print(f'frequencia de respostas corretas: {respostas_certas/total}')
print()
print(f'Abstinencias: {abstinencias}')
print(f'frequencia de abstinencias: {abstinencias/total}')
print()
print(f'Respostas erradas: {total - respostas_certas - abstinencias}')
print(f'frequencia de respostas erradas: {(total - respostas_certas - abstinencias)/total}')

Respostas corretas: 22
frequencia de respostas corretas: 0.9565217391304348

Abstinencias: 0
frequencia de abstinencias: 0.0

Respostas erradas: 1
frequencia de respostas erradas: 0.043478260869565216


# total de respostas corretas

In [ ]:
print(f'respostas corretas demo: {total_demo}')
print(f'freq respostas corretas demo: {total_demo/(len(demo)*10)}')
print(f'abstinencias demo: {abstinencias_demo}')

respostas corretas demo: 172
freq respostas corretas demo: 0.6370370370370371
abstinencias demo: 46


In [ ]:
print(f'respostas corretas video: {total_video}')
print(f'freq respostas corretas video: {total_video/(len(video)*10)}')
print(f'abstinencias video: {abstinencias_video}')

respostas corretas video: 162
freq respostas corretas video: 0.7043478260869566
abstinencias video: 19
